In [1]:
import pandas as pd

In [3]:
bike = pd.read_csv("DS_lv1/bike.csv")
bike.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [21]:
bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


## Q1. 일일권으로 자전거를 대여한 숫자인 casual 변수의 제1사분위수와 제3사분위수를 계산하시오
.quantile() 메서드 활용

In [8]:
a = bike["casual"].quantile(0.25)
a

4.0

In [9]:
b = bike["casual"].quantile(0.75)
b

49.0

## Q2. casual의 사분위범위(IQR)을 산출하여 stat_iqr이라는 객체에 저장하시오

In [13]:
stat_iqr = b - a
stat_iqr

45.0

## Q3. casual 변수의 이상치는 몇개인지 확인하시오
제 3사분위수보다 1.5IQR을 넘어가는 숫자를 이상치로 간주한다

In [20]:
sum(bike["casual"] > (b +1.5*stat_iqr))

749

## Q4. 총 자전거 대여 숫자인 count 변수의 이상치를 확인하고 해당 변수의 이상치가 있는 row를 추출하여 bike_out 객체에 저장하고, 이상치를 제외한 row를 추출하여 bike_in 객체에 저장하시오.
이상치는 제 1사분위수보다 1.5IQR 작거나 제 3사분위수보다 1.5IQR 큰걸 기준

In [48]:
a = bike["count"].quantile(0.25)
b = bike["count"].quantile(0.75)
stat_iqr = b - a
stat_iqr

242.0

In [50]:
condi_1 = bike["count"] < (a -1.5*stat_iqr)
condi_2 = bike["count"] > (b +1.5*stat_iqr)
    
bike_out = bike.loc[condi_1 | condi_2, ]
print(len(bike_out))

300


In [56]:
condi_3 = bike["count"] >= (a -1.5*stat_iqr)
condi_4 = bike["count"] <= (b +1.5*stat_iqr)
    
bike_in = bike.loc[condi_3 & condi_4, ]
# bike_out = bike.loc[~(condi_1 & condi_2), ]  #앞에 ~붙이면 반전
print(len(bike_in))
bike_in.head(2)

10586


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,bike_out
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,False
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,False


In [32]:
bike["count"][0]

16

In [57]:
# 첨도(확률분포의 뾰족한 정도)
bike["casual"].kurt()

7.551629305632764

In [58]:
# 왜도(자료의 치우침 정도)
bike["casual"].skew()

2.4957483979812567

In [60]:
bike_sub = bike[['temp', 'atemp', 'casual', 'registered']]
bike_sub.head(2)

,temp,atemp,casual,registered
0,9.84,14.395,3,13
1,9.02,13.635,8,32


In [61]:
# 상관계수
bike_sub.corr()

,temp,atemp,casual,registered
temp,1.000000,0.984948,0.467097,0.318571
atemp,0.984948,1.000000,0.462067,0.314635
casual,0.467097,0.462067,1.000000,0.497250
registered,0.318571,0.314635,0.497250,1.000000


In [62]:
bike_sub.corr(method = 'kendall')

,temp,atemp,casual,registered
temp,1.000000,0.899929,0.377493,0.231688
atemp,0.899929,1.000000,0.375837,0.230423
casual,0.377493,0.375837,1.000000,0.570630
registered,0.231688,0.230423,0.570630,1.000000


In [63]:
bike_sub.corr(method = 'spearman')

,temp,atemp,casual,registered
temp,1.000000,0.987128,0.573034,0.352174
atemp,0.987128,1.000000,0.571588,0.350577
casual,0.573034,0.571588,1.000000,0.775785
registered,0.352174,0.350577,0.775785,1.000000


In [65]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau

In [66]:
stat, p = pearsonr(bike_sub["temp"], bike_sub["casual"])
print(stat) # 상관계수
print(p) # p-value. 유의수준보다 작으면 귀무가설을 기각하고 대립가설을 채택함.

0.46709706412013274
0.0


In [67]:
dia = pd.read_csv("DS_lv1/diamonds.csv")
dia.head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


In [70]:
dia_E = dia.loc[dia['color'] == "E", ]
dia_H = dia.loc[dia['color'] == "H", ]

In [71]:
print(len(dia_E))
print(len(dia_H))

9797
8304


In [72]:
from scipy.stats import bartlett
bartlett(dia_E["price"], dia_H["price"])
#p-value가 유의수준 0.05보다 작음 : 등분산 검정 결과 분산은 같지않다.

BartlettResult(statistic=484.16407135967086, pvalue=2.6525226756537817e-107)

In [76]:
from scipy.stats import chi2_contingency
df = pd.read_csv("DS_lv1/traffic_accident.txt", sep = "\t")
df

,seq,sex,serious_yn,age
0,1,M,N,3
1,2,M,N,6
2,3,M,N,4
3,4,M,N,4
4,5,F,N,3
...,...,...,...,...
1062,1063,F,N,6
1063,1064,M,N,4
1064,1065,M,N,6
1065,1066,F,N,6


In [77]:
chi2_contingency(pd.crosstab(df["serious_yn"], df["sex"]))
# 두번째가 p-value다. 0.05보다 크므로 독립임

(3.7895026717172966,
 0.051574970410524265,
 1,
 array([[286.99343955, 679.00656045],
        [ 30.00656045,  70.99343955]]))

In [78]:
chi2_contingency(pd.crosstab(df["serious_yn"], df["sex"]), correction = False)

(4.248074934656112,
 0.03929484927552679,
 1,
 array([[286.99343955, 679.00656045],
        [ 30.00656045,  70.99343955]]))

# Quiz - bank

In [80]:
import pandas as pd
df = pd.read_csv("DS_lv1/bank.csv", sep = ";")
df.head(2)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no


## Q1. 열살 단위로 구분하는 age_g 변수를 새로 생성하시오
ex) 30살 -> 30, 49 -> 40

In [88]:
age_g = df["age"] // 10 * 10
# age_g = (df["age"] / 10).astype("int")
df["age_g"] = age_g
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,age_g
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no,30
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no,30
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no,30
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no,30
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no,30
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no,50
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no,50
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no,20


## Q2. 연령(age)과 예금잔고(balance)간 Pearson's 상관분석을 실시하고 그 결과를 확인하시오.

In [89]:
from scipy.stats import pearsonr

In [90]:
stat, p = pearsonr(df["age"], df["balance"])
print(stat) #상관계수
print(p) #p-value
#상관이 거의 없다시피 한 수준이다.(독립)

0.08382014224477764
1.658480874647685e-08


## Q3. 혼인상태(marital)와 이직의사 여부(y) 간 독립성 검정을 실시하고 그 결과를 확인하시오

In [87]:
chi2_contingency(pd.crosstab(df["marital"], df["y"]))
# 독립이다...?

(19.03005634409817,
 7.373535401823763e-05,
 2,
 array([[ 467.15328467,   60.84671533],
        [2474.67374475,  322.32625525],
        [1058.17297058,  137.82702942]]))

In [91]:
from scipy.stats import ttest_1samp

In [4]:
df = pd.read_excel("DS_lv1/iris_xlsx.xlsx")
df.head(2)

NameError: name 'pd' is not defined

In [104]:
df["Sepal.Length"].mean()

5.843333333333335

In [109]:
stat, p = ttest_1samp(df["Sepal.Length"], popmean = 4)
print(stat)
print(p)

27.263680640799215
8.764592435410748e-60


In [119]:
# 모평균 t-검정
stat, p = ttest_1samp(df["Sepal.Length"], popmean = 5.8)
# popmean : 모평균이 5.843333과 가까워지면서, p-value를 보면 유의미하게 다르다고 하기 어려운 상태가 됨.
print(stat)
print(p)

0.6409183514112012
0.5225602746220779


In [120]:
from scipy.stats import ttest_rel # 대응표본
from scipy.stats import ttest_ind # 독립표본

In [121]:
ttest_rel(df["Sepal.Length"], df["Sepal.Width"])

Ttest_relResult(statistic=34.81519257082818, pvalue=1.8495537347987982e-73)

In [124]:
ttest_ind(df["Sepal.Length"], df["Sepal.Width"], equal_var= True) #등분산 가정

Ttest_indResult(statistic=36.463283934449116, pvalue=7.027919392009369e-112)

In [125]:
ttest_ind(df["Sepal.Length"], df["Sepal.Width"], equal_var= False) #이분산 가정

Ttest_indResult(statistic=36.46328393444912, pvalue=1.4595425075861685e-96)

In [127]:
stat, p = ttest_ind(df["Sepal.Width"], df["Sepal.Length"]) # 순서를 바꾸면 음수가 나옴 그러나 p-value는 다르지 않음
# 상관계수의 절댓값을 말하고 있다.
print(stat)
print(round(abs(stat),2))

-36.463283934449116
36.46


In [130]:
from scipy.stats import f_oneway
df = pd.read_excel("DS_lv1/iris_xlsx.xlsx")
stat, p = f_oneway(df["Sepal.Length"],
                  df["Sepal.Width"],
                  df["Petal.Length"])

In [132]:
# 일원 분산분석 - 귀무가설 : 전체 집단간 평균이 같다
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
df = pd.read_csv("DS_lv1/diamonds.csv")
formula = "price ~ C(cut)"
lm = ols(formula, df).fit()
print(anova_lm(lm))

               df        sum_sq       mean_sq           F         PR(>F)
C(cut)        4.0  1.104175e+10  2.760436e+09  175.688717  8.428307e-150
Residual  53935.0  8.474314e+11  1.571209e+07         NaN            NaN


In [135]:
bike = pd.read_csv("DS_lv1/bike.csv")
bike.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [137]:
lm1 = ols("temp ~ season", bike).fit()
anova_lm(lm1)

,df,sum_sq,mean_sq,F,PR(>F)
season,1.0,44221.657301,44221.657301,780.591754,5.879560e-166
Residual,10884.0,616594.417651,56.651453,NaN,NaN


In [138]:
lm2 = ols("temp ~ C(season)", bike).fit()
anova_lm(lm2)

,df,sum_sq,mean_sq,F,PR(>F)
C(season),3.0,412885.270005,137628.423335,6040.687453,0.0
Residual,10882.0,247930.804947,22.783570,NaN,NaN


In [140]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [141]:
# 사후검정 : 어느 집단간 차이를 보이는지 검정
dia = pd.read_csv("DS_lv1/diamonds.csv")
posthoc = pairwise_tukeyhsd(df["price"], df["cut"])
print(posthoc)
#reject ; 유의수준 5%기준 귀무가설 기각여부(True는 기각)

     Multiple Comparison of Means - Tukey HSD, FWER=0.05      
 group1   group2   meandiff p-adj    lower      upper   reject
--------------------------------------------------------------
   Fair      Good -429.8933 0.0015  -740.4567 -119.3299   True
   Fair     Ideal -901.2158  0.001 -1180.5785 -621.8531   True
   Fair   Premium  225.4999  0.195   -59.2739  510.2738  False
   Fair Very Good -376.9979 0.0031  -663.8694  -90.1263   True
   Good     Ideal -471.3225  0.001   -642.367 -300.2779   True
   Good   Premium  655.3933  0.001   475.6466  835.1399   True
   Good Very Good   52.8954    0.9  -130.1565  235.9474  False
  Ideal   Premium 1126.7157  0.001  1008.8058 1244.6257   True
  Ideal Very Good  524.2179  0.001   401.3281  647.1078   True
Premium Very Good -602.4978  0.001  -737.2366 -467.7591   True
--------------------------------------------------------------
